In [40]:
!pip3 uninstall xlrd

Found existing installation: xlrd 2.0.1
Uninstalling xlrd-2.0.1:
  Would remove:
    /home/personal/.local/bin/runxlrd.py
    /home/personal/.local/lib/python3.8/site-packages/xlrd-2.0.1.dist-info/*
    /home/personal/.local/lib/python3.8/site-packages/xlrd/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import requests

In [ ]:
xls_path = "data/nombres_por_edad_media.xls"

In [ ]:
xl = pd.ExcelFile(xls_path)
sheets_lst = xl.sheet_names
print(sheets_lst)

['Hombres', 'Mujeres']


In [ ]:
test_name_df = pd.read_excel(
    xls_path, sheet_name = sheets_lst[1], skiprows = 6)

In [ ]:
test_name_df.head(5)

,Orden,Nombre,Frecuencia,Edad Media (*)
0,1,MARIA CARMEN,641875,59.5
1,2,MARIA,578124,48.4
2,3,CARMEN,366259,60.5
3,4,ANA MARIA,269814,53.6
4,5,MARIA PILAR,258063,59.4


In [ ]:
columns = ["name", "freq", "avg_age"]
test_name_df = test_name_df.drop(columns = ["Orden"])
test_name_df.columns = columns
test_name_df["name"] = test_name_df["name"].str.upper().str.strip()

In [ ]:
test_name_df.head()

,name,freq,avg_age
0,MARIA CARMEN,641875,59.5
1,MARIA,578124,48.4
2,CARMEN,366259,60.5
3,ANA MARIA,269814,53.6
4,MARIA PILAR,258063,59.4


In [59]:
def load_and_clean_names_xls(gender_str):
    columns = ["name", "freq", "avg_age"]
    if gender_str == 'M':
        sheet_idx = 0
    elif gender_str == 'F':
        sheet_idx = 1
    else:
        sheet_idx = None
    df = pd.read_excel(
        xls_path, sheet_name = sheets_lst[sheet_idx], skiprows = 6)
    df = df.drop(columns = ["Orden"])
    df.columns = columns
    df["name"] = df["name"].str.upper().str.strip()
    return df

In [60]:
female_name_df = load_and_clean_names_xls("F")
male_name_df = load_and_clean_names_xls("M")

In [61]:
female_name_df.head()

,name,freq,avg_age
0,MARIA CARMEN,641875,59.5
1,MARIA,578124,48.4
2,CARMEN,366259,60.5
3,ANA MARIA,269814,53.6
4,MARIA PILAR,258063,59.4


In [62]:
male_name_df.head()

,name,freq,avg_age
0,ANTONIO,640854,57.1
1,MANUEL,562798,55.7
2,JOSE,547142,62.1
3,FRANCISCO,467933,58.7
4,DAVID,369633,32.9


In [63]:
male_name_df[male_name_df["name"].str.contains("GUSTAVO")]

,name,freq,avg_age
210,GUSTAVO,12785,43.9
458,GUSTAVO ADOLFO,4101,44.0
1201,CARLOS GUSTAVO,1056,44.2
2362,LUIS GUSTAVO,416,37.6
2404,GUSTAVO ALBERTO,404,47.2
...,...,...,...
26387,GUSTAVO ELIAS,20,38.9
26388,GUSTAVO HUMBERTO,20,48.1
26421,HUGO GUSTAVO,20,36.4
26640,LUCAS GUSTAVO,20,28.1


In [64]:
name_df = pd.merge(
    female_name_df.set_index("name"), 
    male_name_df.set_index("name"), 
    on = ["name"],  
    how = "outer",
    suffixes=('_f', '_m'))

In [65]:
name_df.head()

,freq_f,avg_age_f,freq_m,avg_age_m
name,,,,
MARIA CARMEN,641875.0,59.5,NaN,NaN
MARIA,578124.0,48.4,183.0,55.4
CARMEN,366259.0,60.5,NaN,NaN
ANA MARIA,269814.0,53.6,NaN,NaN
MARIA PILAR,258063.0,59.4,NaN,NaN


In [66]:
name_df["freq_f"] = name_df["freq_f"].fillna(0)
name_df["freq_m"] = name_df["freq_m"].fillna(0)
name_df["prob_f"] = name_df["freq_f"] / (name_df["freq_f"] + name_df["freq_m"])
name_df["prob_m"] = name_df["freq_m"] / (name_df["freq_f"] + name_df["freq_m"])
name_df["gender"] = np.where(name_df["prob_f"] >= 0.5, "F", "M")

In [67]:
name_df[(name_df["gender"] == "F")].sort_values(["prob_m"], ascending = False).head(5)

,freq_f,avg_age_f,freq_m,avg_age_m,prob_f,prob_m,gender
name,,,,,,,
XIAOYANG,21.0,39.9,21.0,27.0,0.5,0.5,F
WISAM,55.0,15.3,55.0,16.0,0.5,0.5,F
ERI,22.0,33.0,22.0,24.4,0.5,0.5,F
MANJEET,55.0,40.0,55.0,37.7,0.5,0.5,F
SHU,31.0,30.4,31.0,33.9,0.5,0.5,F


In [68]:
name_basics_df = name_df[["prob_f", "prob_m", "gender"]]

In [69]:
name_basics_df.sample(5)

,prob_f,prob_m,gender
name,,,
CANDELARIA MARGARITA,1.0,0.0,F
MANUELA ELENA,1.0,0.0,F
ENRIKE,0.0,1.0,M
LILIAN CAROLINA,1.0,0.0,F
MARIANA LUMINITA,1.0,0.0,F


In [ ]:
name_basics_df.to_csv("data/names_master.csv")

In [ ]:
#!pip3 install django djangorestframework markdown django-filter

In [ ]:
#!python3 -m django --version

4.0.6


In [ ]:
#!django-admin startproject 

In [ ]:
#!python3 manage.py runserver 0.0.0.0:8000

In [ ]:
#!python manage.py startapp 

In [ ]:
sqliteConnection = sqlite3.connect('db.sqlite3')
cursor = sqliteConnection.cursor()

sqlite_select_Query = """
    SELECT name
    FROM sqlite_master
    WHERE type ='table';
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

('django_migrations',)
('sqlite_sequence',)
('auth_group_permissions',)
('auth_user_groups',)
('auth_user_user_permissions',)
('django_admin_log',)
('django_content_type',)
('auth_permission',)
('auth_group',)
('auth_user',)
('django_session',)
('web_name',)


In [ ]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    PRAGMA table_info([web_name]);
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

(0, 'id', 'integer', 1, None, 1)
(1, 'text', 'varchar(255)', 1, None, 0)
(2, 'prob_f', 'real', 1, None, 0)
(3, 'prob_m', 'real', 1, None, 0)
(4, 'gender', 'varchar(255)', 1, None, 0)
(5, 'pub_date', 'datetime', 0, None, 0)


In [ ]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    SELECT * 
    FROM web_name
    LIMIT 5;
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

(36580, 'MARIA CARMEN', 1.0, 0.0, 'F', None)
(36581, 'MARIA', 0.9996835590784825, 0.0003164409215174639, 'F', None)
(36582, 'CARMEN', 1.0, 0.0, 'F', None)
(36583, 'ANA MARIA', 1.0, 0.0, 'F', None)
(36584, 'MARIA PILAR', 1.0, 0.0, 'F', None)


In [ ]:
cursor = sqliteConnection.cursor()

for index, row in name_basics_df.reset_index().iterrows():
    name = str(row["name"]).replace("'", " ")  
    prob_f = row["prob_f"]
    prob_m = row["prob_m"]
    gender = row["gender"]    

    insert_into_name = """
        INSERT INTO web_name 
            (text, prob_f, prob_m, gender)
            VALUES ('{0}', {1}, {2}, '{3}');
        """.format(name, prob_f, prob_m, gender) 
    if index == 0:
        print("Example query:")
        print(insert_into_name)
    try:
        cursor.execute(insert_into_name)
    except:
        print(insert_into_name)
    sqliteConnection.commit()  

cursor.close()

Example query:

        INSERT INTO web_name 
            (text, prob_f, prob_m, gender)
            VALUES ('MARIA CARMEN', 1.0, 0.0, 'F');
        


In [ ]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    SELECT * 
    FROM web_name;
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

(91625, 'MARIA CARMEN', 1.0, 0.0, 'F', None)
(91626, 'MARIA', 0.9996835590784825, 0.0003164409215174639, 'F', None)
(91627, 'CARMEN', 1.0, 0.0, 'F', None)
(91628, 'ANA MARIA', 1.0, 0.0, 'F', None)
(91629, 'MARIA PILAR', 1.0, 0.0, 'F', None)
(91630, 'LAURA', 1.0, 0.0, 'F', None)
(91631, 'JOSEFA', 1.0, 0.0, 'F', None)
(91632, 'ISABEL', 1.0, 0.0, 'F', None)
(91633, 'MARIA DOLORES', 1.0, 0.0, 'F', None)
(91634, 'MARIA TERESA', 1.0, 0.0, 'F', None)
(91635, 'ANA', 1.0, 0.0, 'F', None)
(91636, 'MARTA', 1.0, 0.0, 'F', None)
(91637, 'CRISTINA', 1.0, 0.0, 'F', None)
(91638, 'MARIA ANGELES', 1.0, 0.0, 'F', None)
(91639, 'LUCIA', 1.0, 0.0, 'F', None)
(91640, 'MARIA ISABEL', 1.0, 0.0, 'F', None)
(91641, 'MARIA JOSE', 1.0, 0.0, 'F', None)
(91642, 'FRANCISCA', 1.0, 0.0, 'F', None)
(91643, 'ANTONIA', 1.0, 0.0, 'F', None)
(91644, 'DOLORES', 0.9998785878665128, 0.00012141213348712204, 'F', None)
(91645, 'SARA', 0.9997927091190714, 0.00020729088092866313, 'F', None)
(91646, 'PAULA', 1.0, 0.0, 'F', None)


In [ ]:
#response = requests.get("http://127.0.0.1:8080/api/names/LUIS/")

In [ ]:
#r.status_code

In [ ]:
#print(r.text)

In [70]:
name_basics_backup_df = name_basics_df

In [71]:
name_basics_df = name_basics_df.reset_index().head()

In [72]:
name_basics_df["lword"] = name_basics_df["name"].str[-1:]
name_basics_df["fword"] = name_basics_df["name"].str[:1]

In [73]:
name_basics_df.head()

,name,prob_f,prob_m,gender,lword,fword
0,MARIA CARMEN,1.000000,0.000000,F,N,M
1,MARIA,0.999684,0.000316,F,A,M
2,CARMEN,1.000000,0.000000,F,N,C
3,ANA MARIA,1.000000,0.000000,F,A,A
4,MARIA PILAR,1.000000,0.000000,F,R,M


In [74]:
def is_vowel(word):
    if word.lower() in "aeiou":
        return 1
    return 0

name_basics_df["lvowel"] = name_basics_df["lword"].apply(is_vowel)
name_basics_df["fvowel"] = name_basics_df["fword"].apply(is_vowel)

In [75]:
name_basics_df.head()

,name,prob_f,prob_m,gender,lword,fword,lvowel,fvowel
0,MARIA CARMEN,1.000000,0.000000,F,N,M,0,0
1,MARIA,0.999684,0.000316,F,A,M,1,0
2,CARMEN,1.000000,0.000000,F,N,C,0,0
3,ANA MARIA,1.000000,0.000000,F,A,A,1,1
4,MARIA PILAR,1.000000,0.000000,F,R,M,0,0


In [76]:
def count_words(name):
    l = len(name.split(" "))   
    return l

name_basics_df["num_words"] = \
    name_basics_df["name"].apply(count_words)

In [77]:
name_basics_df.head()

,name,prob_f,prob_m,gender,lword,fword,lvowel,fvowel,num_words
0,MARIA CARMEN,1.000000,0.000000,F,N,M,0,0,2
1,MARIA,0.999684,0.000316,F,A,M,1,0,1
2,CARMEN,1.000000,0.000000,F,N,C,0,0,1
3,ANA MARIA,1.000000,0.000000,F,A,A,1,1,2
4,MARIA PILAR,1.000000,0.000000,F,R,M,0,0,2


In [78]:
def count_chars(name):
    l = len(name)   
    return l

name_basics_df["num_chars"] = \
    name_basics_df["name"].apply(count_chars)

In [79]:
name_basics_df = name_basics_df.drop(
    columns = ["prob_f", "prob_m"], 
    axis = 1
)

name_basics_df.head()

,name,gender,lword,fword,lvowel,fvowel,num_words,num_chars
0,MARIA CARMEN,F,N,M,0,0,2,12
1,MARIA,F,A,M,1,0,1,5
2,CARMEN,F,N,C,0,0,1,6
3,ANA MARIA,F,A,A,1,1,2,9
4,MARIA PILAR,F,R,M,0,0,2,11
